# Importing Librarires and Data

In [1]:
import pandas as pd

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adilshamim8/salaries-for-data-science-jobs")

print("Path to dataset files:", path)

c:\Users\felip\OneDrive\Documentos\GitHub\Data-Science-Salary-Prediction\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\felip\.cache\kagglehub\datasets\adilshamim8\salaries-for-data-science-jobs\versions\15


In [3]:
df = pd.read_csv(r"C:\Users\felip\.cache\kagglehub\datasets\adilshamim8\salaries-for-data-science-jobs\versions\15\salaries.csv")

In [4]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2025,EX,FT,Head of Data,348516,USD,348516,US,0,US,M
1,2025,EX,FT,Head of Data,232344,USD,232344,US,0,US,M
2,2025,SE,FT,Data Scientist,145400,USD,145400,US,0,US,M
3,2025,SE,FT,Data Scientist,81600,USD,81600,US,0,US,M
4,2025,MI,FT,Engineer,160000,USD,160000,US,100,US,M


# Exploratory Data Analysis

In [5]:
# The table has 151.445  rows and 11 columns, 
df.shape

(151445, 11)

In [13]:
# There isn't any missing values in the dataset, and the data types seems to be right
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151445 entries, 0 to 151444
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   work_year           151445 non-null  int64 
 1   experience_level    151445 non-null  object
 2   employment_type     151445 non-null  object
 3   job_title           151445 non-null  object
 4   salary              151445 non-null  int64 
 5   salary_currency     151445 non-null  object
 6   salary_in_usd       151445 non-null  int64 
 7   employee_residence  151445 non-null  object
 8   remote_ratio        151445 non-null  int64 
 9   company_location    151445 non-null  object
 10  company_size        151445 non-null  object
dtypes: int64(4), object(7)
memory usage: 12.7+ MB


In [20]:
# As usual, there is a perceptible  standart deviation in the salary column
df.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
work_year,151445.0,2024.44,0.67,2020.0,2024.0,2025.0,2025.0,2025.0
salary,151445.0,162837.96,208012.40,14000.0,106000.0,147000.0,199000.0,30400000.0
salary_in_usd,151445.0,157527.46,74150.77,15000.0,105800.0,146100.0,198000.0,800000.0
remote_ratio,151445.0,20.94,40.62,0.0,0.0,0.0,0.0,100.0


In [33]:
df['job_title'].unique()

array(['Head of Data', 'Data Scientist', 'Engineer', 'AI Product Lead',
       'AI Engineer', 'Business Intelligence Engineer', 'Manager',
       'Computer Vision Engineer', 'Data Management', 'Product Manager',
       'Data Engineer', 'Machine Learning Engineer', 'Software Engineer',
       'Data Analyst', 'Research Scientist', 'Developer', 'Associate',
       'Computational Scientist', 'Analyst', 'AI Scientist',
       'DataOps Engineer', 'Prompt Engineer', 'Data Governance Engineer',
       'Analytics Engineer', 'AI Governance Specialist',
       'Backend Engineer', 'DevOps Engineer', 'Systems Engineer',
       'Tech Lead', 'Solutions Engineer', 'Product Analyst', 'Researcher',
       'Executive', 'Director', 'Consultant', 'Solutions Architect',
       'Data Reporting Specialist', 'BI Developer', 'Data Manager',
       'Machine Learning Scientist', 'Data Governance',
       'Applied Scientist', 'Data Architect', 'Statistical Programmer',
       'Data Specialist', 'Technical Architec

In [74]:
df['job_title'].value_counts().head(50)

job_title
Data Scientist                     18751
Software Engineer                  16948
Data Engineer                      16352
Data Analyst                       13779
Engineer                           11004
Machine Learning Engineer           8887
Manager                             7811
Analyst                             5396
Research Scientist                  3460
Product Manager                     2576
Applied Scientist                   2381
Associate                           2379
Data Architect                      2216
Analytics Engineer                  2139
AI Engineer                         2013
Research Engineer                   1737
Consultant                          1363
Data Manager                        1073
Developer                           1040
Data Specialist                      984
Solutions Architect                  976
Business Intelligence Engineer       956
Engineering Manager                  902
Systems Engineer                     854
Archit

In [54]:
df.shape[0]

151445

In [85]:
df[df['job_title'].str.contains('BI', case=False, na=False)]['job_title'].value_counts().head(30)

job_title
Site Reliability Engineer    502
BI Analyst                   447
BI Developer                 306
Power BI Developer           151
BI Engineer                  120
Bioinformatics Scientist      86
Computational Biologist       78
Bioinformatician              48
Power BI                      40
BI Data Analyst               19
Power BI Administrator        18
Big Data Engineer             14
Big Data Developer            11
Power BI Specialist           10
Power BI Expert               10
Bioinformatics Specialist      4
Power BI Architect             2
Power BI Consultant            2
Big Data Architect             2
BI & Data Analyst              1
Big Data Analyst               1
BI Data Engineer               1
Name: count, dtype: int64